## **Extraer datos Generales por dia**

In [0]:
#codigo para guardar todos los insights necesarios para el analisis de datos de SOLO los anuncios
import requests
import pandas as pd
import os

# Leer el token desde el archivo CSV previamente guardado
df_TokenLargoPlazo = pd.read_csv("/dbfs/mnt/Token_LargoPlazo/long_lived_token.csv")
access_token = df_TokenLargoPlazo["access_token"].iloc[0]

# Reemplaza este valor con tu ad_account_id (ID de la cuenta publicitaria de Meta)
ad_account_id = "1952584805183463"

# Crear el directorio '/dbfs:/mnt/processed' si no existe
processed_dir = '/dbfs/mnt/processed/Ads_General_Por_Dia'
if not os.path.exists(processed_dir):
    os.makedirs(processed_dir)

# Función para obtener todos los conjuntos de anuncios de todas las campañas
def obtener_conjuntos_anunciosMeta(access_token, ad_account_id):
    url = f"https://graph.facebook.com/v13.0/act_{ad_account_id}/adsets"
    params = {
        "access_token": access_token,
        "fields": "id,name,status"  # Especifica las columnas que deseas obtener
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()['data']
        df_conjuntosMeta = pd.DataFrame(data)
        return df_conjuntosMeta
    else:
        print(f"Error {response.status_code}: {response.text}")
        return None

# Función para obtener el alcance de un anuncio mediante el endpoint de insights
def obtener_alcance_anuncio(access_token, ad_id):
    url = f"https://graph.facebook.com/v13.0/{ad_id}/insights"
    params = {
        "access_token": access_token,
        "fields": "ad_id,ad_name,adset_name,campaign_name,date_start,date_stop,reach,impressions,frequency,spend,clicks,cost_per_ad_click,inline_link_clicks,conversion_rate_ranking,cpc,cpp,cpm,actions,ad_click_actions,quality_ranking,conversions,buying_type",  # Especifica las métricas que deseas obtener
        "time_increment":1,
        "level": "ad"  # Nivel de la métrica, en este caso es a nivel de anuncio
    }

    
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()['data']
        df_insights = pd.DataFrame(data)
        return df_insights
    else:
        print(f"Error {response.status_code}: {response.text}")
        return None

# Obtener todos los conjuntos de anuncios
df_conjuntos = obtener_conjuntos_anunciosMeta(access_token, ad_account_id)

if df_conjuntos is not None:
    all_insights = []
    # Obtener y guardar el alcance y otras métricas para cada anuncio en los conjuntos de anuncios
    for adset_id in df_conjuntos['id']:
        url = f"https://graph.facebook.com/v13.0/{adset_id}/ads"
        params = {
            "access_token": access_token,
            "fields": "id,name"  # Obtener solo los IDs de los anuncios
        }
        response = requests.get(url, params=params)
        if response.status_code == 200:
            ads_data = response.json()['data']
            for ad in ads_data:
                ad_id = ad['id']
                df_anuncio_insights = obtener_alcance_anuncio(access_token, ad_id)
                if df_anuncio_insights is not None:
                    all_insights.append(df_anuncio_insights)

    # Unir todos los DataFrames de insights en uno solo
    if all_insights:
        df_all_insights = pd.concat(all_insights, ignore_index=True)
        # Guardar todos los insights en un solo CSV
        df_all_insights.to_csv(f"{processed_dir}/anuncios_insights_general_por_dia.csv", index=False)
        print("Todos los insights de los anuncios generales por hora guardados exitosamente.")
        
    else:
        print("No se encontraron insights de anuncios.")
else:
    print("No se encontraron conjuntos de anuncios o limite de solicitud excedido.")
#revisar cuales archivos existen dentro de la ruta
dbutils.fs.ls("/mnt/processed")


Error 400: {"error":{"message":"User request limit reached","type":"OAuthException","is_transient":false,"code":17,"error_subcode":2446079,"error_user_title":"La cuenta publicitaria tiene demasiadas llamadas a la API","error_user_msg":"Se han realizado demasiadas llamadas desde esta cuenta publicitaria. Espera unos instantes y vuelve a intentarlo.","fbtrace_id":"A17mD7z0dAeNRc2BXMON6T-"}}
No se encontraron conjuntos de anuncios o limite de solicitud excedido.


[FileInfo(path='dbfs:/mnt/processed/Ads_General_Por_Dia/', name='Ads_General_Por_Dia/', size=0, modificationTime=1731353128000),
 FileInfo(path='dbfs:/mnt/processed/Ads_General_Total/', name='Ads_General_Total/', size=0, modificationTime=1731429232000)]

In [0]:
dbutils.fs.ls("/mnt/processed/Ads_General_Por_Dia/")

[]

In [0]:
#revisar el contenido de cada columna
import pandas as pd
#crear dataframe para leer el csv
df = pd.read_csv("/dbfs/mnt/processed/Ads_General_Por_Dia/anuncios_insights_general_por_dia.csv")

# Ver las columnas del DataFrame
display(df.columns)
display(df)

Index(['ad_id', 'ad_name', 'adset_name', 'campaign_name', 'date_start',
       'date_stop', 'reach', 'impressions', 'frequency', 'spend', 'clicks',
       'conversion_rate_ranking', 'cpc', 'cpp', 'cpm', 'actions',
       'quality_ranking', 'buying_type', 'inline_link_clicks'],
      dtype='object')

ad_id,ad_name,adset_name,campaign_name,date_start,date_stop,reach,impressions,frequency,spend,clicks,conversion_rate_ranking,cpc,cpp,cpm,actions,quality_ranking,buying_type,inline_link_clicks
120211236488430669,Test_AdVideo_LosArtistas,Test_Group_VideosGeneralesPt2,Test_All_Nov_Camp,2024-11-06,2024-11-06,77,81,1.051948,2.8,1,UNKNOWN,2.8,36.363636,34.567901,"[{'action_type': 'post_engagement', 'value': '10'}, {'action_type': 'page_engagement', 'value': '10'}, {'action_type': 'video_view', 'value': '10'}]",UNKNOWN,AUCTION,null
120211236488430669,Test_AdVideo_LosArtistas,Test_Group_VideosGeneralesPt2,Test_All_Nov_Camp,2024-11-07,2024-11-07,268,335,1.25,11.86,9,UNKNOWN,1.317778,44.253731,35.402985,"[{'action_type': 'post_engagement', 'value': '60'}, {'action_type': 'page_engagement', 'value': '60'}, {'action_type': 'video_view', 'value': '59'}, {'action_type': 'post_reaction', 'value': '1'}]",UNKNOWN,AUCTION,null
120211236488430669,Test_AdVideo_LosArtistas,Test_Group_VideosGeneralesPt2,Test_All_Nov_Camp,2024-11-08,2024-11-08,52,57,1.096154,1.38,0,UNKNOWN,null,26.538462,24.210526,"[{'action_type': 'post_engagement', 'value': '6'}, {'action_type': 'page_engagement', 'value': '6'}, {'action_type': 'video_view', 'value': '6'}]",UNKNOWN,AUCTION,null
120211236488430669,Test_AdVideo_LosArtistas,Test_Group_VideosGeneralesPt2,Test_All_Nov_Camp,2024-11-09,2024-11-09,46,53,1.152174,1.41,0,UNKNOWN,null,30.652174,26.603774,"[{'action_type': 'post_engagement', 'value': '10'}, {'action_type': 'page_engagement', 'value': '10'}, {'action_type': 'video_view', 'value': '10'}]",UNKNOWN,AUCTION,null
120211236488430669,Test_AdVideo_LosArtistas,Test_Group_VideosGeneralesPt2,Test_All_Nov_Camp,2024-11-10,2024-11-10,75,96,1.28,3.9,1,UNKNOWN,3.9,52.0,40.625,"[{'action_type': 'post_engagement', 'value': '20'}, {'action_type': 'page_engagement', 'value': '20'}, {'action_type': 'video_view', 'value': '20'}]",UNKNOWN,AUCTION,null
120211236578610669,Test_AdVideo_PreNavidad,Test_Group_VideosGeneralesPt2,Test_All_Nov_Camp,2024-11-06,2024-11-06,24,29,1.208333,1.08,3,UNKNOWN,0.36,45.0,37.241379,"[{'action_type': 'post_engagement', 'value': '7'}, {'action_type': 'page_engagement', 'value': '7'}, {'action_type': 'video_view', 'value': '7'}]",UNKNOWN,AUCTION,null
120211236578610669,Test_AdVideo_PreNavidad,Test_Group_VideosGeneralesPt2,Test_All_Nov_Camp,2024-11-07,2024-11-07,133,199,1.496241,11.7,6,UNKNOWN,1.95,87.969925,58.79397,"[{'action_type': 'post_engagement', 'value': '20'}, {'action_type': 'page_engagement', 'value': '20'}, {'action_type': 'video_view', 'value': '20'}]",UNKNOWN,AUCTION,null
120211236578610669,Test_AdVideo_PreNavidad,Test_Group_VideosGeneralesPt2,Test_All_Nov_Camp,2024-11-08,2024-11-08,33,42,1.272727,2.26,1,UNKNOWN,2.26,68.484848,53.809524,"[{'action_type': 'post_engagement', 'value': '7'}, {'action_type': 'page_engagement', 'value': '7'}, {'action_type': 'video_view', 'value': '7'}]",UNKNOWN,AUCTION,null
120211236578610669,Test_AdVideo_PreNavidad,Test_Group_VideosGeneralesPt2,Test_All_Nov_Camp,2024-11-09,2024-11-09,11,12,1.090909,0.6,0,UNKNOWN,null,54.545455,50.0,"[{'action_type': 'post_engagement', 'value': '3'}, {'action_type': 'page_engagement', 'value': '3'}, {'action_type': 'video_view', 'value': '3'}]",UNKNOWN,AUCTION,null
120211236578610669,Test_AdVideo_PreNavidad,Test_Group_VideosGeneralesPt2,Test_All_Nov_Camp,2024-11-10,2024-11-10,48,65,1.354167,3.59,2,UNKNOWN,1.795,74.791667,55.230769,"[{'action_type': 'post_engagement', 'value': '11'}, {'action_type': 'page_engagement', 'value': '11'}, {'action_type': 'video_view', 'value': '11'}]",UNKNOWN,AUCTION,null
